In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [3]:
# Load the .csv dataset.
file_path = "../Resources/Use me 2020 NFL 3rd & Long Raw Dataset.csv"
df = pd.read_csv(file_path)
df

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,Yards,...,IsTwoPointConversion,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards
0,2020122602,12/26/2020,2,9,44,LV,MIA,2,10,-9,...,0,0,NaN,18,OPP,0,NaN,0,NaN,0
1,2020122602,12/26/2020,2,9,3,LV,MIA,3,19,18,...,0,0,LEFT GUARD,27,OPP,0,NaN,0,NaN,0
2,2020122500,12/25/2020,3,1,17,MIN,NO,3,9,0,...,0,0,NaN,33,OPP,0,NO,1,DEFENSIVE HOLDING,0
3,2020122500,12/25/2020,3,1,12,MIN,NO,1,10,0,...,0,0,NaN,18,OPP,1,NO,1,DEFENSIVE PASS INTERFERENCE,17
4,2020122500,12/25/2020,3,1,6,MIN,NO,1,1,-3,...,0,0,LEFT GUARD,1,OPP,0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46184,2020091300,9/13/2020,4,1,38,ATL,SEA,2,9,2,...,0,0,NaN,20,OPP,0,NaN,0,NaN,0
46185,2020091300,9/13/2020,4,5,18,NaN,NaN,0,0,0,...,0,0,NaN,0,OPP,0,NaN,0,NaN,0
46186,2020091300,9/13/2020,4,2,0,ATL,SEA,1,10,1,...,0,0,NaN,21,OPP,0,NaN,0,NaN,0
46187,2020091300,9/13/2020,4,2,0,NaN,NaN,0,0,0,...,0,0,NaN,0,OPP,0,NaN,0,NaN,0


In [4]:

df.columns

Index(['GameId', 'GameDate', 'Quarter', 'Minute', 'Second', 'OffenseTeam',
       'DefenseTeam', 'Down', 'ToGo', 'Yards', 'Yards (+,-)',
       'Successful 3rd Down Y/N', 'YardLine', 'SeriesFirstDown', 'NextScore',
       'Description', 'TeamWin', 'SeasonYear', 'Formation', 'PlayType',
       'IsRush', 'IsPass', 'IsIncomplete', 'IsTouchdown', 'PassType', 'IsSack',
       'IsChallenge', 'IsChallengeReversed', 'Challenger', 'IsMeasurement',
       'IsInterception', 'IsFumble', 'IsPenalty', 'IsTwoPointConversion',
       'IsTwoPointConversionSuccessful', 'RushDirection', 'YardLineFixed',
       'YardLineDirection', 'IsPenaltyAccepted', 'PenaltyTeam', 'IsNoPlay',
       'PenaltyType', 'PenaltyYards'],
      dtype='object')

In [5]:
df["OffenseTeam"].unique()

array(['LV', 'MIN', 'TB', 'ARI', 'DEN', 'MIA', nan, 'NYJ', 'CLE', 'NYG',
       'BAL', 'CIN', 'HOU', 'CHI', 'IND', 'PIT', 'DAL', 'WAS', 'SEA',
       'LA', 'CAR', 'TEN', 'BUF', 'KC', 'GB', 'ATL', 'NE', 'JAX', 'SF',
       'LAC', 'PHI', 'DET', 'NO'], dtype=object)

In [94]:
#Keep
df = df[['Quarter', 'Down' , 'ToGo', 'Successful 3rd Down Y/N' , 'Formation','PlayType',]]
df.head()

,Quarter,Down,ToGo,Successful 3rd Down Y/N,Formation,PlayType
0,2,2,10,4th Down,SHOTGUN,SACK
1,2,3,19,4th Down,SHOTGUN,RUSH
2,3,3,9,4th Down,SHOTGUN,PASS
3,3,1,10,4th Down,UNDER CENTER,PASS
4,3,1,1,4th Down,UNDER CENTER,RUSH


In [95]:
df.isna().sum()

Quarter                    0
Down                       0
ToGo                       0
Successful 3rd Down Y/N    0
Formation                  0
PlayType                   0
dtype: int64

In [96]:
#Drop all null values
df = df.dropna()
df.head()

,Quarter,Down,ToGo,Successful 3rd Down Y/N,Formation,PlayType
0,2,2,10,4th Down,SHOTGUN,SACK
1,2,3,19,4th Down,SHOTGUN,RUSH
2,3,3,9,4th Down,SHOTGUN,PASS
3,3,1,10,4th Down,UNDER CENTER,PASS
4,3,1,1,4th Down,UNDER CENTER,RUSH


In [97]:
playlist=df['PlayType'].unique().tolist()
playlist2=['PASS','RUSH','FUMBLE','SACK','SCRAMBLE']

In [98]:
q4_df=\
df.loc[(df['Quarter']==4)&\
       (df['Down']== 3)&\
      (df['PlayType'].isin(playlist2))\
      ].reset_index()

In [99]:
q4_df['PlayType'].value_counts()

PASS        1192
RUSH         434
SACK         137
SCRAMBLE      74
Name: PlayType, dtype: int64

In [100]:
q4_df

,index,Quarter,Down,ToGo,Successful 3rd Down Y/N,Formation,PlayType
0,11,4,3,10,N,SHOTGUN,PASS
1,25,4,3,2,Y,UNDER CENTER,RUSH
2,32,4,3,3,Y,SHOTGUN,RUSH
3,55,4,3,7,N,SHOTGUN,PASS
4,67,4,3,10,N,SHOTGUN,PASS
...,...,...,...,...,...,...,...
1832,45977,4,3,3,Y,SHOTGUN,PASS
1833,45984,4,3,9,N,SHOTGUN,PASS
1834,45993,4,3,19,N,SHOTGUN,PASS
1835,46090,4,3,1,N,SHOTGUN,RUSH


# Split Data into Training and Testing

In [101]:
# Create our features
y = q4_df["Successful 3rd Down Y/N"]
X = q4_df.drop("Successful 3rd Down Y/N", axis=1)
X = pd.get_dummies(X)

In [102]:
# Create our target
y = y.replace({"N":0, "Y":1})
y.value_counts()

0    1091
1     746
Name: Successful 3rd Down Y/N, dtype: int64

In [103]:
X.describe()

,index,Quarter,Down,ToGo,Formation_NO HUDDLE,Formation_NO HUDDLE SHOTGUN,Formation_SHOTGUN,Formation_UNDER CENTER,PlayType_PASS,PlayType_RUSH,PlayType_SACK,PlayType_SCRAMBLE
count,1837.000000,1837.0,1837.0,1837.000000,1837.000000,1837.000000,1837.000000,1837.000000,1837.000000,1837.000000,1837.000000,1837.000000
mean,22401.446924,4.0,3.0,6.880240,0.009799,0.085465,0.765378,0.139358,0.648884,0.236255,0.074578,0.040283
std,13458.814066,0.0,0.0,5.152133,0.098528,0.279649,0.423878,0.346414,0.477449,0.424896,0.262781,0.196676
min,11.000000,4.0,3.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10378.000000,4.0,3.0,3.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,21799.000000,4.0,3.0,6.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,34170.000000,4.0,3.0,10.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,46183.000000,4.0,3.0,34.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [104]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Logistic Regression

In [105]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier

LogisticRegression(random_state=1)

In [106]:
# Train the data
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [107]:
predictions = classifier.predict(X_test)

In [108]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.6130434782608696

# Balanced Random Forest Classifier

In [109]:
X_train.head()

,index,Quarter,Down,ToGo,Formation_NO HUDDLE,Formation_NO HUDDLE SHOTGUN,Formation_SHOTGUN,Formation_UNDER CENTER,PlayType_PASS,PlayType_RUSH,PlayType_SACK,PlayType_SCRAMBLE
812,19360,4,3,13,0,0,1,0,1,0,0,0
1388,34339,4,3,1,0,0,0,1,0,1,0,0
1205,29780,4,3,19,0,0,1,0,1,0,0,0
1282,31497,4,3,9,0,0,1,0,0,1,0,0
957,22739,4,3,18,0,0,1,0,0,1,0,0


In [110]:
# Resample the training data with the BalancedRandomForestClassifier
# Create a random forest classifier.
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
brf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=78) 
# Fitting the model
brf_model = brf_model.fit(X_train_scaled, y_train)
y_pred = brf_model.predict(X_test_scaled)

In [111]:
# Calculated the balanced accuracy score
# Calculating the accuracy score.
acc_score = balanced_accuracy_score(y_test,y_pred)
acc_score

0.5927782960199004

In [112]:
# Display the confusion matrix
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,160,108
Actual 1,79,113


In [113]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
# Print the imbalanced classification report
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Balanced Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_pred))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,160,108
Actual 1,79,113


Balanced Accuracy Score : 0.5927782960199004
Classification Report
              precision    recall  f1-score   support

           0       0.67      0.60      0.63       268
           1       0.51      0.59      0.55       192

    accuracy                           0.59       460
   macro avg       0.59      0.59      0.59       460
weighted avg       0.60      0.59      0.60       460



In [114]:
# List the features sorted in descending order by feature importance
importances = brf_model.feature_importances_
sorted(zip(importances, X.columns), reverse=True)

[(0.6527211219017393, 'index'),
 (0.23666284427335588, 'ToGo'),
 (0.053748737628900575, 'PlayType_SACK'),
 (0.01482406622747291, 'PlayType_PASS'),
 (0.009212016758956526, 'Formation_SHOTGUN'),
 (0.008919657863690119, 'PlayType_SCRAMBLE'),
 (0.008804987498256915, 'PlayType_RUSH'),
 (0.006540628243898529, 'Formation_NO HUDDLE SHOTGUN'),
 (0.006323961810505609, 'Formation_UNDER CENTER'),
 (0.002241977793223837, 'Formation_NO HUDDLE'),
 (0.0, 'Quarter'),
 (0.0, 'Down')]

# Easy Ensemble AdaBoost Classifier

In [115]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
classifier = EasyEnsembleClassifier(n_estimators=100)

classifier.fit(X_train_scaled, y_train)
predictions = classifier.predict(X_test_scaled)

In [116]:
# Calculated the balanced accuracy score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
acc_score = accuracy_score(y_test, y_pred)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.5934782608695652


In [117]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
   cm, index=["Actual 0", "Actual 1"],
   columns=["Predicted 0", "Predicted 1"]
)
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,160,108
Actual 1,79,113


In [118]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report(y_test, y_pred))

Classification Report
              precision    recall  f1-score   support

           0       0.67      0.60      0.63       268
           1       0.51      0.59      0.55       192

    accuracy                           0.59       460
   macro avg       0.59      0.59      0.59       460
weighted avg       0.60      0.59      0.60       460



# Combination (Over and Under)

In [119]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({0: 191, 1: 221})

In [120]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [121]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.5927782960199004

In [122]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[142, 126],
       [ 48, 144]], dtype=int64)

In [123]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.75      0.53      0.75      0.62      0.63      0.39       268
          1       0.53      0.75      0.53      0.62      0.63      0.41       192

avg / total       0.66      0.62      0.66      0.62      0.63      0.40       460



# SMOTE Oversampling

In [124]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(
   X_train, y_train)

Counter(y_resampled)

Counter({0: 823, 1: 823})

In [125]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [126]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [127]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[142, 126],
       [ 48, 144]], dtype=int64)

In [128]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.75      0.53      0.75      0.62      0.63      0.39       268
          1       0.53      0.75      0.53      0.62      0.63      0.41       192

avg / total       0.66      0.62      0.66      0.62      0.63      0.40       460

